[View in Colaboratory](https://colab.research.google.com/github/opooladz/ece_210_final_project/blob/master/210_final_project_convnet.ipynb)

In [0]:
# Import Libs 
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math
#from  scipy.ndimage import convolve1d

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
# Keras try model 
def try_model(model, data, name, epochs=200, verbose=1):  
  model.compile(optimizer='nadam',
            loss='categorical_crossentropy',
            metrics=[ 'accuracy'])

  model.summary()  
  history = model.fit(data['x_train'], data['y_train'], 
            epochs=epochs, batch_size=64, verbose=verbose, 
            validation_data=(data['x_val'], data['y_val']))  
  score = model.evaluate(data['x_test'], data['y_test'])
  print('Test accuracy: {}'.format(score))
  
#   out_path = 'serialized_models/{}.h5'.format(name)  
#   model.save(out_path)
  
#   with open('serialized_models/{}_history.pickle'.format(name), 'wb') as handle:
#     pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
  return history

In [0]:
# Comms Stuff Will add more later
def qammod(m,y):
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  x = a + 1.*1j*b
  # lets hold x in a np array([real,imag])
  # x = np.array([a,1.*b]).T
  return [x , y] 

def qammod2(m,size,trials):
  """
  m - highest integer for the randint
  """
  y = np.random.randint(m,size=(trials,size))
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  #x = a + 1.*1j*b
  # lets hold x in a dp array([real,imag])
  #x = np.array([a,1.*b])
  x = np.dstack((a,b))
  return [x , y]                        

def channel2(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  
  [x,y] = qammod2(m,sequenceLen,trials)
  
  h = [1., 0.5, 0.3, -0.13, 0.2, 0, 0.4]
  #h = np.random.randn(1,chanelLen)
  #h = [1]
  #print(x.shape)
#   xh = np.empty((sequenceLen+len(h)-1,2))
#   xh[:,0] = np.convolve(x[:,0],h)   
#   xh[:,1] = np.convolve(x[:,1],h)   
  #xh = convolve1d(x,h,axis = 1)
  xh = np.apply_along_axis(lambda m: np.convolve(m,h[0:chanelLen],mode="full"),axis = 1,arr=x)
  #print(xh.shape)
  SNR = 10**(SNRdB/10)
  sym_noise_pow = np.var(xh)/SNR
  sym_noise_scale_fact = np.sqrt(sym_noise_pow/2)
  if(Noise):
    AWGN = sym_noise_scale_fact*np.random.normal(1, size=xh.shape) 
    xh = xh + AWGN
  #return [xhN.T , np.repeat(y,2) ]
  return [xh ,y ]
  
                        
data={}
def generateData2(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  [data['x_train'],data['y_train']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data['x_val'],data['y_val']] = channel2(m=m,sequenceLen=sequenceLen,trials=int(trials*0.5),Noise= Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data['x_test'],data['y_test']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)

In [39]:
# Visulizing the data

m = 4         # order of modulation
#seqLen = 64    # length of one trial must be square powers of 2 to make sense see below 
trials = 1500   # number of trials 
Noise = 1
SNRdB = 20
chanelLen = 4


x = 4
square_power_of_two = 2**(2*x)
seqLen = square_power_of_two  
print(seqLen)

generateData2(m,seqLen,trials,Noise,SNRdB,chanelLen)
print(data['x_train'].shape) # data  # axis 1 should be 2x seqLen because of conv with a 2tap filter
print(data['y_train'].shape) # labels
#print(data['x_train']) # so u can see the data 

# tmp = (data['x_train'][:,:-1].reshape(trials,seqLen))

# #plot 1
# plt.scatter(np.real(tmp[0][:]),np.imag(tmp[0][:]))   
#                                          # I only plot the first 10 points from 
#                                          # the sequence because as time goes on 
#                                          # the data becomes less dependent on the
#                                          # previous symbols. Even 10 is a lot. 
# plt.title('4-QAM Mod With Multi-Channel AWGN plot 1')
# plt.show()

#plot 2
# Just a pure 16 qam with 1 point on each constalation point 
# clearly we have 16 classes
# [xtmp,ytmp]=  qammod(16,data['y_train'])
# plt.scatter(xtmp[:,0],xtmp[:,1])
# plt.title('16-QAM Mod plot 2')
# plt.show()



256
(1500, 259, 2)
(1500, 256)


In [40]:
# Some keras stuffs
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten, AveragePooling2D, MaxPooling2D, Dropout, BatchNormalization
from keras.layers import Reshape, Permute
from keras import regularizers
import keras
ykeys = {'y_train','y_val','y_test'}
xkeys = {'x_train','x_val','x_test'}

  
y_train = keras.utils.to_categorical(data['y_train'] , num_classes=m)
y_val = keras.utils.to_categorical(data['y_val'] , num_classes=m)
y_test = keras.utils.to_categorical(data['y_test'] , num_classes=m)

print(data['x_train'].shape)
print(data['x_test'].shape)
print(y_train.shape)
print(data['x_val'].shape)


default_data = {
  'x_train': data['x_train'],
  'x_val': data['x_val'],
  'x_test': data['x_test'],
  'y_train': y_train,
  'y_val': y_val,
  'y_test': y_test,
}


(1500, 259, 2)
(1500, 259, 2)
(1500, 256, 4)
(750, 259, 2)


In [41]:
from keras.layers import Reshape,Dense,Embedding,Conv1D, LSTM,CuDNNLSTM, TimeDistributed, GaussianNoise, Softmax,Flatten
from keras import regularizers

modCNN = Sequential()
modCNN.add(Conv1D(20,kernel_size=(80), input_shape=(seqLen+chanelLen-1,2) ))
modCNN.add(BatchNormalization())
modCNN.add(Conv1D(20,kernel_size=(80), input_shape=(seqLen+chanelLen-1,2) ))
modCNN.add(BatchNormalization())
modCNN.add(Flatten())
modCNN.add(Dense(seqLen*m,bias_regularizer=regularizers.l2(0.005)))
modCNN.add(Reshape(target_shape = (seqLen,m)))
modCNN.add(BatchNormalization())
#modCNN.add(Dropout(0.4))
modCNN.add(Softmax(axis=2))
try_model(modCNN, default_data, 'conv_lstmx', epochs=50)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_32 (Conv1D)           (None, 180, 20)           3220      
_________________________________________________________________
batch_normalization_44 (Batc (None, 180, 20)           80        
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 101, 20)           32020     
_________________________________________________________________
batch_normalization_45 (Batc (None, 101, 20)           80        
_________________________________________________________________
flatten_15 (Flatten)         (None, 2020)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 1024)              2069504   
_________________________________________________________________
reshape_16 (Reshape)         (None, 256, 4)            0         
__________

1500/1500 [==============================] - 1s 400us/step - loss: 0.4046 - acc: 0.9568 - val_loss: 0.5259 - val_acc: 0.8726
Epoch 7/50
1500/1500 [==============================] - 1s 406us/step - loss: 0.3670 - acc: 0.9677 - val_loss: 0.4810 - val_acc: 0.8894
Epoch 8/50
1500/1500 [==============================] - 1s 411us/step - loss: 0.3364 - acc: 0.9749 - val_loss: 0.4443 - val_acc: 0.9018
Epoch 9/50
1500/1500 [==============================] - 1s 404us/step - loss: 0.3108 - acc: 0.9801 - val_loss: 0.4141 - val_acc: 0.9112
Epoch 10/50
1500/1500 [==============================] - 1s 394us/step - loss: 0.2887 - acc: 0.9843 - val_loss: 0.3866 - val_acc: 0.9196
Epoch 11/50
1500/1500 [==============================] - 1s 396us/step - loss: 0.2696 - acc: 0.9874 - val_loss: 0.3644 - val_acc: 0.9265
Epoch 12/50
1500/1500 [==============================] - 1s 402us/step - loss: 0.2528 - acc: 0.9895 - val_loss: 0.3449 - val_acc: 0.9324
Epoch 13/50
1500/1500 [==============================] -

Epoch 18/50
1500/1500 [==============================] - 1s 391us/step - loss: 0.1793 - acc: 0.9964 - val_loss: 0.2697 - val_acc: 0.9535
Epoch 19/50
1500/1500 [==============================] - 1s 419us/step - loss: 0.1701 - acc: 0.9969 - val_loss: 0.2607 - val_acc: 0.9554
Epoch 20/50
1500/1500 [==============================] - 1s 405us/step - loss: 0.1621 - acc: 0.9973 - val_loss: 0.2529 - val_acc: 0.9577
Epoch 21/50
1500/1500 [==============================] - 1s 405us/step - loss: 0.1543 - acc: 0.9977 - val_loss: 0.2457 - val_acc: 0.9603
Epoch 22/50
1500/1500 [==============================] - 1s 399us/step - loss: 0.1473 - acc: 0.9980 - val_loss: 0.2389 - val_acc: 0.9616
Epoch 23/50
1500/1500 [==============================] - 1s 406us/step - loss: 0.1406 - acc: 0.9983 - val_loss: 0.2320 - val_acc: 0.9635
Epoch 24/50
1500/1500 [==============================] - 1s 413us/step - loss: 0.1344 - acc: 0.9984 - val_loss: 0.2261 - val_acc: 0.9653
Epoch 25/50
1500/1500 [==================

1500/1500 [==============================] - 1s 394us/step - loss: 0.1089 - acc: 0.9992 - val_loss: 0.2006 - val_acc: 0.9715
Epoch 30/50
1500/1500 [==============================] - 1s 397us/step - loss: 0.1044 - acc: 0.9993 - val_loss: 0.1966 - val_acc: 0.9719
Epoch 31/50
1500/1500 [==============================] - 1s 403us/step - loss: 0.1006 - acc: 0.9994 - val_loss: 0.1924 - val_acc: 0.9729
Epoch 32/50
1500/1500 [==============================] - 1s 400us/step - loss: 0.0968 - acc: 0.9994 - val_loss: 0.1884 - val_acc: 0.9741
Epoch 33/50
1500/1500 [==============================] - 1s 408us/step - loss: 0.0933 - acc: 0.9995 - val_loss: 0.1847 - val_acc: 0.9746
Epoch 34/50
1500/1500 [==============================] - 1s 394us/step - loss: 0.0899 - acc: 0.9995 - val_loss: 0.1807 - val_acc: 0.9756
Epoch 35/50
1500/1500 [==============================] - 1s 393us/step - loss: 0.0867 - acc: 0.9996 - val_loss: 0.1771 - val_acc: 0.9764
Epoch 36/50
1500/1500 [==============================

1500/1500 [==============================] - 1s 395us/step - loss: 0.0705 - acc: 0.9999 - val_loss: 0.1589 - val_acc: 0.9799
Epoch 42/50
1500/1500 [==============================] - 1s 393us/step - loss: 0.0683 - acc: 0.9999 - val_loss: 0.1561 - val_acc: 0.9804
Epoch 43/50
1500/1500 [==============================] - 1s 401us/step - loss: 0.0662 - acc: 0.9999 - val_loss: 0.1534 - val_acc: 0.9808
Epoch 44/50
1500/1500 [==============================] - 1s 410us/step - loss: 0.0642 - acc: 0.9999 - val_loss: 0.1510 - val_acc: 0.9811
Epoch 45/50
1500/1500 [==============================] - 1s 420us/step - loss: 0.0622 - acc: 0.9999 - val_loss: 0.1483 - val_acc: 0.9819
Epoch 46/50
1500/1500 [==============================] - 1s 402us/step - loss: 0.0604 - acc: 1.0000 - val_loss: 0.1455 - val_acc: 0.9821
Epoch 47/50
1500/1500 [==============================] - 1s 403us/step - loss: 0.0585 - acc: 1.0000 - val_loss: 0.1434 - val_acc: 0.9827
Epoch 48/50
1500/1500 [==============================